<a href="https://colab.research.google.com/github/Gyaneswar33/FMML__LABS__AND__PROJECTS/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

1st answer

In [ ]:
Yes, averaging validation accuracy across multiple splits, such as in k-fold cross-validation, tends to give more consistent and reliable results. Here's why:

1. Reduction of Variance: When you use a single train-validation split, the performance of your model may vary significantly depending on how the data is split, especially if the dataset is small or imbalanced. Averaging across multiple splits reduces the variance in the estimate of your model’s accuracy, giving a more stable performance measure.


2. Better Generalization: By training and validating on different portions of the data in each split, you ensure that your model is evaluated on various samples, thus providing a more generalizable measure of performance.


3. Handling Bias in Split: Certain splits may either overestimate or underestimate performance due to the specific distribution of training and validation sets. Averaging over multiple splits balances out these biases.


4. Improved Robustness: If the data contains outliers or noise, averaging across multiple splits helps to mitigate the impact of these anomalies on the final accuracy result.



In short, using multiple splits (like cross-validation) and averaging their accuracies provides a more robust and consistent estimate of the model's performance compared to using a single validation split.

2nd answer

In [ ]:
Yes, averaging validation accuracy across multiple splits (e.g., via k-fold cross-validation) generally gives a more accurate estimate of the model's performance on unseen data, or test accuracy, for several reasons:

1. Reduced Overfitting to a Single Split

When using a single train-test split, the model may overfit to the particular characteristics of that split, leading to an optimistic or pessimistic estimate of the test accuracy. By averaging across multiple splits, you reduce the chance of overfitting to any specific validation set, making the estimate closer to how the model will perform on entirely new data.

2. Exposure to More Data Variability

Each fold in k-fold cross-validation exposes the model to different portions of the data for training and testing. This means the final averaged validation accuracy reflects the model's performance on a broader range of examples, making it more representative of general test accuracy.

3. Reduction of Sampling Bias

With a single split, the data might accidentally be divided in a way that is not representative of the true data distribution (e.g., an imbalance of classes or features in training vs. validation sets). Multiple splits counterbalance such biases, as the model gets evaluated on various test sets, leading to a more accurate estimate of performance on unseen data.

4. Better Estimate for Small Datasets

In small datasets, a single train-test split can give misleading results, as a small validation set may not fully capture the data's diversity. Cross-validation maximizes the use of available data for both training and validation, leading to a more accurate estimate of the test accuracy.

5. Statistical Robustness

The average of multiple validation results is less prone to extreme values (high or low) that can arise due to noise or outliers in a single test set, making it statistically more robust and closer to the true test performance.

In summary, averaging validation accuracy across multiple splits typically provides a more reliable and accurate estimate of test accuracy compared to using a single train-test split, especially in scenarios where the data size is small or where there is significant variability in the data distribution.

3rd answer

In [ ]:
The number of iterations in a cross-validation process (or the number of folds, in the case of k-fold cross-validation) plays a significant role in the accuracy and reliability of the performance estimate. Here’s how the number of iterations (or splits) affects the estimate:

1. Higher Number of Iterations (More Folds)

Reduced Variance: As the number of iterations increases, the model is trained and validated on a larger variety of training and validation sets. This helps reduce the variance in the performance estimate, making it more stable and reliable.

Improved Use of Data: More iterations lead to more refined estimates, as the model is evaluated on more unique subsets of the data. In extreme cases, leave-one-out cross-validation (LOO-CV), where each iteration leaves out one sample for validation and uses the rest for training, gives a very fine-grained estimate. However, this can be computationally expensive.

More Representative Estimate: Higher iteration counts tend to better represent the performance of the model across the entire dataset, especially when the dataset is small. More iterations mean the model has more opportunities to learn from various subsets of the data.


2. Trade-off with Computation

Increased Computational Cost: Increasing the number of iterations (folds) makes the cross-validation process more computationally intensive. For example, k = 10 folds means the model is trained and validated 10 times, while k = 100 would mean 100 times. Beyond a certain point, the additional gain in estimate accuracy may not justify the increased computational cost.

Diminishing Returns: While more iterations generally lead to a better estimate, there are diminishing returns beyond a certain point. For example, going from k = 5 to k = 10 might significantly improve the estimate, but going from k = 10 to k = 50 may yield only marginal improvements.


3. Small Datasets

Smaller Validation Set Sizes: In scenarios with small datasets, increasing the number of iterations (folds) can lead to smaller validation sets for each fold. For example, in leave-one-out cross-validation (LOO-CV), there’s only one data point in the validation set for each iteration, which can lead to high variance in the individual accuracy estimates, even though the overall average might be unbiased.

Balancing Bias and Variance: For small datasets, too few iterations may give a biased estimate (depending heavily on the specific data in each fold), but too many iterations may also result in estimates with high variance (due to very small validation sets). Therefore, it’s important to balance the number of iterations for small datasets.


4. Common Practice

5- or 10-Fold Cross-Validation: In practice, 5-fold or 10-fold cross-validation is commonly used. These values strike a good balance between computational efficiency and accuracy of the performance estimate. Research shows that 10-fold cross-validation often offers a good balance between variance and bias in the estimate.


Summary

More iterations (higher k) tend to give a better estimate of test accuracy because the model is evaluated on a broader range of data splits, leading to lower variance and more consistent results.

However, there are diminishing returns after a certain point. For most practical purposes, 5- to 10-fold cross-validation is typically sufficient.

If you increase the number of iterations too much (like in leave-one-out cross-validation), the computational cost rises significantly, and in some cases, the estimate's variance might increase due to small validation sets in each fold.


Therefore, while more iterations generally improve the accuracy of the estimate, the benefit decreases after a certain number, and balancing computational cost and accuracy is key.

4th answer

In [ ]:
Increasing the number of iterations (or folds) in cross-validation can help to some extent when dealing with a very small training or validation dataset, but it has limitations. Here's a more detailed explanation:

1. Increasing Iterations Can Help:

Maximizing Data Usage: When you have a small dataset, one of the main benefits of increasing the number of iterations (or folds) is that more data is used for training and validation across the different folds. For example, in 10-fold cross-validation, 90% of the data is used for training and 10% for validation in each iteration. This means every data point is used for both training and validation at some point, which helps make the most out of a small dataset.

More Reliable Performance Estimate: By averaging performance across more iterations, you can reduce the bias that might result from the small size of the validation set in a single split. The more folds you have, the more robust the performance estimate tends to be.


2. Limitations of Increasing Iterations with Small Datasets:

Smaller Validation Sets in Each Fold: As you increase the number of folds, the validation set in each fold becomes smaller. For example, if you use 10-fold cross-validation on a small dataset, each validation set will contain only 10% of the data. In extreme cases, such as leave-one-out cross-validation (LOO-CV), the validation set contains only a single sample. While this maximizes the training set size, it can lead to high variance in individual accuracy estimates for each fold, since evaluating the model on a small validation set can be highly sensitive to the specific data points included.

Overfitting Risk: For very small datasets, even with a larger number of iterations, the model may still overfit to the limited training data. If the dataset is not representative or if certain patterns appear due to chance, the model may learn noise instead of meaningful generalizable patterns, and this overfitting will affect the performance estimate.

Computational Cost: While increasing the number of folds helps reduce variance and maximize data usage, this also comes at a computational cost, especially with complex models. The smaller the dataset, the more folds you'd need to reduce bias, but this might result in a higher number of model training and validation cycles, which can become computationally expensive.


3. Possible Solutions Beyond Just Increasing Iterations:

Data Augmentation: Instead of simply increasing the number of folds, consider using data augmentation (for tasks like image classification) or synthetic data generation techniques, where new training examples are artificially generated. This increases the effective size of the training set.

Transfer Learning: If the dataset is very small, you can use transfer learning, especially for tasks like computer vision and natural language processing. In this case, you start with a pre-trained model and fine-tune it on your small dataset, which can help mitigate the lack of training data.

Regularization: Applying stronger regularization techniques (e.g., L2 regularization, dropout) can help prevent overfitting when working with small datasets.

Ensemble Methods: Ensemble learning (combining predictions from multiple models) can help stabilize the variance in small datasets, improving generalization even when training data is scarce.


4. Practical Considerations:

5- to 10-Fold Cross-Validation as a Good Compromise: For small datasets, using 5- or 10-fold cross-validation strikes a balance between having enough data in the validation sets and maximizing the data used for training. This is typically more practical than increasing iterations excessively.

Use Nested Cross-Validation for Hyperparameter Tuning: If you need to perform hyperparameter tuning on small datasets, consider using nested cross-validation. This approach allows for proper selection of hyperparameters while avoiding overfitting to the training set, providing a better estimate of test accuracy.


Conclusion:

Yes, increasing iterations helps with small datasets, as it reduces bias and variance in the performance estimate and makes better use of the limited data. However, it’s not a complete solution.

Smaller validation sets can result from increasing iterations, leading to higher variance in individual accuracy estimates and potentially overfitting on the small training sets.

Therefore, you should balance the number of iterations with other techniques like data augmentation, regularization, and transfer learning to handle very small datasets more effectively.